<a href="https://colab.research.google.com/github/t-zakharenko/ML-course-May-2025/blob/main/HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_TZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [4]:
raw_df = pd.read_csv('customer_segmentation_train.csv', index_col=0)

In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 693.3+ KB


In [6]:
'''
Заповнюємо пропущені значення
Провела додатковий аналіз даних, з'ясувала, що по колонці 'Ever_Married' мода для людей молодше 33 років - 'No', а для інших - 'Yes'.
Так само різна мода була по колонці 'Graduated', для людей молодше і старше 23 років.

'Work_Experience' заповнила нулем, тому що думаю, що ця колонка могла залишатися незаповненою у випадку, коли людина дійсно ще не мала досвіду роботи.

Spending_Score (text) заміняю на 'Spending_Score_coded', числові значення 1, 2, 3

'''
raw_df['Ever_Married'] = raw_df.apply(
    lambda row: 'No' if pd.isna(row['Ever_Married']) and row['Age'] < 33
    else ('Yes' if pd.isna(row['Ever_Married']) else row['Ever_Married']),
    axis=1
)

raw_df['Graduated'] = raw_df.apply(
    lambda row: 'No' if pd.isna(row['Graduated']) and row['Age'] < 23
    else ('Yes' if pd.isna(row['Graduated']) else row['Graduated']),
    axis=1
)

raw_df['Profession'] = raw_df['Profession'].fillna('Unknown')
raw_df['Var_1'] = raw_df['Var_1'].fillna('Unknown')

raw_df['Work_Experience'] = raw_df['Work_Experience'].fillna(0)
raw_df['Family_Size'] = raw_df['Family_Size'].fillna(raw_df['Family_Size'].median())

# Spending_Score заміняємо на 1-2-3
raw_df['Spending_Score_coded'] = raw_df['Spending_Score'].map({
    'Low': 1,
    'Average': 2,
    'High': 3
})
raw_df = raw_df.drop(columns=['Spending_Score'])


In [7]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Gender                8068 non-null   object 
 1   Ever_Married          8068 non-null   object 
 2   Age                   8068 non-null   int64  
 3   Graduated             8068 non-null   object 
 4   Profession            8068 non-null   object 
 5   Work_Experience       8068 non-null   float64
 6   Family_Size           8068 non-null   float64
 7   Var_1                 8068 non-null   object 
 8   Segmentation          8068 non-null   object 
 9   Spending_Score_coded  8068 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 693.3+ KB


In [8]:
# Розбиття на тренувальну і тестувальну вибірку, де в тесті 20%.
cat_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession',
             'Var_1']
num_cols = ['Age', 'Work_Experience', 'Family_Size', 'Spending_Score_coded']
target_col = 'Segmentation'


X = raw_df[cat_cols + num_cols].copy()
y = raw_df[target_col].copy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [9]:
cat_feature_indices = [
    idx
    for idx, col_name in enumerate(X_train.columns)
    if col_name in cat_cols
]
cat_feature_indices

[0, 1, 2, 3, 4]

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train[num_cols])

X_train[num_cols] = scaler.transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

X_train.describe().round(1)

,Age,Work_Experience,Family_Size,Spending_Score_coded
count,6454.0,6454.0,6454.0,6454.0
mean,0.4,0.2,0.2,0.3
std,0.2,0.2,0.2,0.4
min,0.0,0.0,0.0,0.0
25%,0.2,0.0,0.1,0.0
50%,0.3,0.1,0.2,0.0
75%,0.5,0.3,0.4,0.5
max,1.0,1.0,1.0,1.0


In [22]:
from imblearn.over_sampling import SMOTENC

In [39]:
sm = SMOTENC(random_state=42, categorical_features=cat_feature_indices)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

In [40]:
y_resampled.value_counts()

,count
Segmentation,
A,1814
B,1814
C,1814
D,1814


In [41]:
from imblearn.combine import SMOTETomek

In [47]:
smo_tek = SMOTETomek(random_state=42)
X_smotek, y_smotek = smo_tek.fit_resample(X_resampled[num_cols], y_resampled)

In [48]:
y_smotek.value_counts()

,count
Segmentation,
D,1726
C,1705
A,1690
B,1685


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [51]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(X_train[cat_cols])

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [52]:
def encode_dataframe(df, encoder, cat_cols):
    encoded = encoder.transform(df[cat_cols])
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_cols), index=df.index)
    df = df.drop(columns=cat_cols)
    df = pd.concat([df, encoded_df], axis=1)
    return df

In [58]:
X_train_encoded = encode_dataframe(X_train, encoder, cat_cols)
X_resampled_encoded = encode_dataframe(X_resampled, encoder, cat_cols)
X_smotek_encoded = X_smotek.copy()

X_test_encoded = encode_dataframe(X_test, encoder, cat_cols)

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

In [56]:
def train_and_evaluate(X_train, y_train, X_test, y_test, name):
    model = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"===== {name} =====")
    print(classification_report(y_test, preds))

In [62]:
train_and_evaluate(X_train_encoded, y_train, X_test_encoded, y_test, "Original Data")
train_and_evaluate(X_resampled_encoded, y_resampled, X_test_encoded, y_test, "SMOTENC")

X_smotek_encoded, y_smotek = smo_tek.fit_resample(X_resampled_encoded, y_resampled) #повторно роблю, бо зверху без категоріальних
train_and_evaluate(X_smotek_encoded, y_smotek, X_test_encoded, y_test, "SMOTE-Tomek")

===== Original Data =====
              precision    recall  f1-score   support

           A       0.42      0.45      0.43       394
           B       0.41      0.14      0.21       372
           C       0.49      0.64      0.55       394
           D       0.64      0.75      0.69       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.47      1614
weighted avg       0.49      0.51      0.48      1614

===== SMOTENC =====
              precision    recall  f1-score   support

           A       0.42      0.48      0.45       394
           B       0.41      0.21      0.28       372
           C       0.50      0.61      0.55       394
           D       0.66      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.50      0.50      0.49      1614
weighted avg       0.51      0.52      0.50      1614

===== SMOTE-Tomek =====
              precision    recall  f1-score   support

     

**Висновок:**
1. Напишіть, яку метрику ви обрали для порівняння моделей.
- Я б обрала метрику f1-score, macro average і, як додаткову -
precision
2. Яка модель найкраща?
- найкращою виглядає модель SMOTENC, в неї найкращі показники метрик f1-score і precision, хоча різниця дуже мала.
3. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?
- можливо, балансування классів не вплинуло на результат. Додаткові дані (Томек) не додали суттєвої інформації для навчання моделі.